# Análisis pensiones básico

Este cuaderno resume cada bloque de `pensiones_basico.py`. Ejecuta las celdas en orden para descargar, limpiar, analizar y exportar los datos.


## 1. Librerías y configuración

Importamos todas las dependencias requeridas. `%matplotlib inline` asegura que las gráficas se muestren dentro del notebook.


In [ ]:
%matplotlib inline

import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import entropy as shannon_entropy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.inspection import permutation_importance

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot


## 2. Descarga del dataset público

Definimos la URL del recurso en `datos.gov.co` y descargamos todas las páginas en lotes de 50 000 registros.


In [ ]:
BASE = "https://www.datos.gov.co"
RESOURCE = "uawh-cjvi"
URL = f"{BASE}/resource/{RESOURCE}.json"

try:
    total_filas = int(requests.get(f"{URL}?$select=count(*)", timeout=60).json()[0]["count"])
except Exception:
    total_filas = None

print("Total reportado:", total_filas)

Lista_paginas = []
limit = 50000
offset = 0

while True:
    params = {"$limit": limit, "$offset": offset}
    r = requests.get(URL, params=params, timeout=120)
    r.raise_for_status()
    pagina = r.json()
    if not pagina:
        break
    Lista_paginas.append(pd.DataFrame(pagina))
    offset += limit
    print(f"Descargadas: {offset} filas…")
    time.sleep(0.3)

if Lista_paginas:
    df = pd.concat(Lista_paginas, ignore_index=True)
else:
    df = pd.DataFrame()

df.head()


## 3. Conversión de tipos y limpieza básica

Normalizamos la columna `fecha`, limpiamos `valor_unidad` y rellenamos valores faltantes con forward-fill + interpolación.


In [ ]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

df["valor_unidad"] = (
    df["valor_unidad"]
      .astype(str)
      .str.replace(r"[^\d\-,\.]", "", regex=True)
      .str.replace(",", ".", regex=False)
      .astype(float)
)

print(df.dtypes)
print(df.isnull().sum())

df["valor_unidad"] = df["valor_unidad"].ffill().interpolate()


## 4. Exploración rápida de calidad de datos

Calculamos porcentaje de nulos, cardinalidad y ejemplos de frecuencia por entidad y fondo.


In [ ]:
nulls = df.isna().mean().sort_values(ascending=False).mul(100).round(2)
print(nulls)

cardinalidad = df.nunique(dropna=True).sort_values(ascending=False)
print(cardinalidad)

print("Valores únicos en nombre_entidad:", df["nombre_entidad"].dropna().unique()[:10])
print("Valores únicos en nombre_fondo:", df["nombre_fondo"].dropna().unique()[:10])
print(df["nombre_entidad"].value_counts(dropna=False).head(10))
print(df["nombre_fondo"].value_counts(dropna=False).head(20))


## 5. Exportes base y diccionarios de códigos

Guardamos un CSV simplificado y construimos diccionarios `dict_entidad` y `dict_fondo`.


In [ ]:
Path("data/raw").mkdir(parents=True, exist_ok=True)

df_clean = df.drop(columns=["codigo_entidad", "codigo_patrimonio"], errors="ignore")
df_clean.to_csv("data/raw/pensionesLimpio.csv", index=False)

if "codigo_entidad" in df.columns:
    dict_entidad = (
        df[["nombre_entidad", "codigo_entidad"]]
          .drop_duplicates()
          .set_index("nombre_entidad")["codigo_entidad"]
          .to_dict()
    )
    df[["nombre_entidad", "codigo_entidad"]].drop_duplicates().to_csv(
        "data/raw/entidad_codigo.csv", index=False
    )
else:
    dict_entidad = {}

if "codigo_patrimonio" in df.columns:
    dict_fondo = (
        df[["nombre_fondo", "codigo_patrimonio"]]
          .drop_duplicates()
          .set_index("nombre_fondo")["codigo_patrimonio"]
          .to_dict()
    )
    df[["nombre_fondo", "codigo_patrimonio"]].drop_duplicates().to_csv(
        "data/raw/fondos_codigo.csv", index=False
    )
else:
    dict_fondo = {}

list(dict_entidad.items())[:5], list(dict_fondo.items())[:5]


## 6. Normalización de nombres y relaciones código ↔ nombre

Eliminamos espacios repetidos y verificamos si la relación entre códigos y nombres es uno a uno.


In [ ]:
for columna in ["nombre_entidad", "nombre_fondo"]:
    if columna in df.columns:
        df[columna] = (
            df[columna]
              .astype(str)
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
        )

print(df[["nombre_entidad", "nombre_fondo"]].nunique())

if "codigo_entidad" in df.columns:
    print(df.groupby("codigo_entidad")["nombre_entidad"].nunique().sort_values(ascending=False).head())
    print(df.groupby("nombre_entidad")["codigo_entidad"].nunique().sort_values(ascending=False).head())

if "codigo_patrimonio" in df.columns:
    print(df.groupby("codigo_patrimonio")["nombre_fondo"].nunique().sort_values(ascending=False).head())
    print(df.groupby("nombre_fondo")["codigo_patrimonio"].nunique().sort_values(ascending=False).head())


## 7. Detección y limpieza de duplicados

Revisamos duplicados exactos y duplicados por combinación entidad–fondo–fecha.


In [ ]:
print("=== ANÁLISIS DE DUPLICADOS ===")
duplicados = df.duplicated().sum()
print(f"Filas duplicadas exactas: {duplicados}")
if duplicados > 0:
    df = df.drop_duplicates()
    print(f"Filas tras limpieza exacta: {len(df)}")

duplicados_conceptuales = df.duplicated(subset=["nombre_entidad", "nombre_fondo", "fecha"]).sum()
print(f"Duplicados entidad-fondo-fecha: {duplicados_conceptuales}")
if duplicados_conceptuales > 0:
    df = df.drop_duplicates(subset=["nombre_entidad", "nombre_fondo", "fecha"], keep="first")
    print(f"Filas tras limpieza conceptual: {len(df)}")


## 8. Outliers en `valor_unidad`

Aplicamos el criterio IQR para marcar valores extremos en una columna booleana `es_outlier`.


In [ ]:
Q1 = df["valor_unidad"].quantile(0.25)
Q3 = df["valor_unidad"].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = df[(df["valor_unidad"] < limite_inferior) | (df["valor_unidad"] > limite_superior)]

print(f"Límite inferior: {limite_inferior:.2f}")
print(f"Límite superior: {limite_superior:.2f}")
print(f"Total de outliers: {len(outliers)}")

df["es_outlier"] = False
df.loc[outliers.index, "es_outlier"] = True


## 9. Optimización de tipos y variables derivadas

Convertimos columnas a categorías y creamos variables temporales (`año`, `mes`, `trimestre`) y `tipo_fondo`.


In [ ]:
df["nombre_entidad"] = df["nombre_entidad"].astype("category")
df["nombre_fondo"] = df["nombre_fondo"].astype("category")
df["es_outlier"] = df["es_outlier"].astype(bool)

df["año"] = df["fecha"].dt.year
df["mes"] = df["fecha"].dt.month
df["trimestre"] = df["fecha"].dt.quarter

def clasificar_fondo(nombre):
    nombre = str(nombre).lower()
    if "cesantia" in nombre:
        return "Cesantías"
    if "pension" in nombre:
        return "Pensiones"
    if "alternativo" in nombre:
        return "Alternativo"
    return "Otros"

df["tipo_fondo"] = df["nombre_fondo"].apply(clasificar_fondo).astype("category")
df[["año", "mes", "trimestre", "tipo_fondo"]].head()


## 10. Guardar subconjuntos por entidad y fondo

`guardar_subset` filtra y exporta CSV específicos para cada entidad y fondo relevante.


In [ ]:
def guardar_subset(dataframe, columna, valores, ruta_salida):
    if isinstance(valores, (list, tuple, set)):
        df_subset = dataframe.loc[dataframe[columna].isin(valores)].copy()
    else:
        df_subset = dataframe.loc[dataframe[columna].eq(valores)].copy()
    if columna in df_subset.columns:
        df_subset = df_subset.drop(columns=[columna])
    df_subset.to_csv(ruta_salida, index=False)
    print(ruta_salida, df_subset.shape)
    return df_subset

df_skandia = guardar_subset(df, "nombre_entidad", "Skandia Afp - Accai S.A.", "data/raw/skandia.csv")
df_skandia_cesantias_largo_plazo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Cesantias Largo Plazo", "data/raw/skandia_fondo_cesantias_largo_plazo.csv")
df_skandia_cesantias_corto_plazo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Cesantias Corto Plazo", "data/raw/skandia_fondo_cesantias_corto_plazo.csv")
df_skandia_pensiones_moderado = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Moderado", "data/raw/skandia_fondo_pensiones_moderado.csv")
df_skandia_pensiones_conservador = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Conservador", "data/raw/skandia_fondo_pensiones_conservador.csv")
df_skandia_pensiones_mayor_riesgo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Mayor Riesgo", "data/raw/skandia_fondo_pensiones_mayor_riesgo.csv")
df_skandia_pensiones_retiro_programado = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Retiro Programado", "data/raw/skandia_fondo_pensiones_retiro_programado.csv")
df_skandia_pensiones_alternativo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Alternativo", "data/raw/skandia_fondo_pensiones_alternativo.csv")

df_proteccion = guardar_subset(df, "nombre_entidad", '"Proteccion"', "data/raw/proteccion.csv")
df_porvenir = guardar_subset(df, "nombre_entidad", '"Porvenir"', "data/raw/porvenir.csv")
df_colfondos = guardar_subset(df, "nombre_entidad", '"Colfondos S.A." Y "Colfondos"', "data/raw/colfondos.csv")


## 11. Gráficas comparativas entre entidades

Graficamos y guardamos comparaciones del valor unidad por tipo de fondo y entidad en `data/graficas_comparativas`.


In [ ]:
def graficar_comparacion_entidades_por_fondo(fondos_a_comparar, titulo_base, nombre_archivo):
    plt.figure(figsize=(14, 8))
    colores = ['blue', 'red', 'green', 'orange', 'purple', 'brown']
    for i, (entidad, df_fondo) in enumerate(fondos_a_comparar.items()):
        if len(df_fondo) > 0:
            color = colores[i % len(colores)]
            plt.plot(df_fondo['fecha'], df_fondo['valor_unidad'], label=entidad, color=color, linewidth=2, alpha=0.8)
    plt.title(f'{titulo_base} - Comparación por Entidad')
    plt.xlabel('Fecha')
    plt.ylabel('Valor Unidad')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)
    plt.savefig(f"data/graficas_comparativas/{nombre_archivo}.png", dpi=300, bbox_inches='tight')
    plt.close()

fondos_moderado = {
    'Skandia': df_skandia_pensiones_moderado,
    'Protección': df_proteccion_pensiones_moderado,
    'Porvenir': df_porvenir_pensiones_moderado,
    'Colfondos': df_colfondos_pensiones_moderado
}
graficar_comparacion_entidades_por_fondo(fondos_moderado, "Fondo de Pensiones Moderado", "comparacion_pensiones_moderado")


## 12. Evolución y correlación por entidad

Visualizamos la evolución normalizada (base 100) y la correlación de retornos diarios entre fondos de la misma entidad.


In [ ]:
def evolucion_todos_fondos_entidad(entidad_nombre, dataframes_fondos):
    plt.figure(figsize=(16, 10))
    colores = {
        'Cesantías Largo Plazo': 'blue',
        'Cesantías Corto Plazo': 'lightblue',
        'Pensiones Moderado': 'green',
        'Pensiones Conservador': 'darkgreen',
        'Pensiones Mayor Riesgo': 'red',
        'Pensiones Retiro Programado': 'orange',
        'Pensiones Alternativo': 'purple'
    }
    for fondo_nombre, color in colores.items():
        if fondo_nombre in dataframes_fondos and len(dataframes_fondos[fondo_nombre]) > 0:
            df_temp = dataframes_fondos[fondo_nombre]
            valor_base = df_temp['valor_unidad'].iloc[0]
            serie_norm = df_temp['valor_unidad'] / valor_base * 100
            plt.plot(df_temp['fecha'], serie_norm, label=fondo_nombre, color=color, linewidth=2, alpha=0.7)
    plt.title(f'Evolución de Todos los Fondos - {entidad_nombre} (Base 100)')
    plt.xlabel('Fecha')
    plt.ylabel('Valor Normalizado')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)
    plt.savefig(f'data/graficas_comparativas/evolucion_todos_fondos_{entidad_nombre.lower()}.png', dpi=300, bbox_inches='tight')
    plt.close()

def matriz_correlacion_fondos(entidad_nombre, dataframes_fondos):
    datos_correlacion = {}
    for fondo_nombre, df_temp in dataframes_fondos.items():
        if len(df_temp) > 0:
            serie = df_temp.set_index('fecha')['valor_unidad'].sort_index()
            returns = serie.pct_change().dropna()
            datos_correlacion[fondo_nombre] = returns
    df_corr = pd.DataFrame(datos_correlacion)
    matriz_corr = df_corr.corr()
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(matriz_corr, dtype=bool))
    sns.heatmap(matriz_corr, mask=mask, annot=True, cmap='coolwarm', center=0, square=True, fmt='.2f', cbar_kws={'label': 'Coeficiente'})
    plt.title(f'Matriz de Correlación - {entidad_nombre}')
    plt.tight_layout()
    Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)
    plt.savefig(f'data/graficas_comparativas/correlacion_{entidad_nombre.lower()}.png', dpi=300, bbox_inches='tight')
    plt.close()
    return matriz_corr

entidades_dataframes = {
    'Skandia': {
        'Cesantías Largo Plazo': df_skandia_cesantias_largo_plazo,
        'Cesantías Corto Plazo': df_skandia_cesantias_corto_plazo,
        'Pensiones Moderado': df_skandia_pensiones_moderado,
        'Pensiones Conservador': df_skandia_pensiones_conservador,
        'Pensiones Mayor Riesgo': df_skandia_pensiones_mayor_riesgo,
        'Pensiones Retiro Programado': df_skandia_pensiones_retiro_programado,
        'Pensiones Alternativo': df_skandia_pensiones_alternativo
    },
    'Protección': {
        'Cesantías Largo Plazo': df_proteccion_cesantias_largo_plazo,
        'Cesantías Corto Plazo': df_proteccion_cesantias_corto_plazo,
        'Pensiones Moderado': df_proteccion_pensiones_moderado,
        'Pensiones Conservador': df_proteccion_pensiones_conservador,
        'Pensiones Mayor Riesgo': df_proteccion_pensiones_mayor_riesgo,
        'Pensiones Retiro Programado': df_proteccion_pensiones_retiro_programado,
        'Pensiones Alternativo': df_proteccion_pensiones_alternativo
    }
}

evolucion_todos_fondos_entidad('Skandia', entidades_dataframes['Skandia'])
matriz_correlacion_fondos('Skandia', entidades_dataframes['Skandia'])


## 13. Variables `lag`

Creamos lags cada 30 días hasta un año y un bloque compacto con lags estándar.


In [ ]:
for lag in range(1, len(df) // 30, 30):
    df[f'lag_{lag}'] = df['valor_unidad'].shift(lag)

lags = [1, 7, 30, 90, 180, 365]
lag_columns = {f'lag_{lag}': df['valor_unidad'].shift(lag) for lag in lags}
lag_df = pd.DataFrame(lag_columns)
df = pd.concat([df, lag_df], axis=1)
df.filter(like='lag_').head()


## 14. Exploración descriptiva y visualizaciones clave

Calculamos estadísticas básicas y generamos gráficas principales, guardándolas en `data/graficas_comparativas`.


In [ ]:
Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)

print(df[['valor_unidad']].describe())
print(df.groupby('tipo_fondo', observed=True)['valor_unidad'].describe())

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
df['valor_unidad'].hist(bins=50, alpha=0.7)
plt.title('Distribución de valor_unidad')
plt.subplot(1, 2, 2)
df['valor_unidad'].plot(kind='density')
plt.title('Densidad de valor_unidad')
plt.tight_layout()
plt.savefig('data/graficas_comparativas/distribucion_y_densidad_valor_unidad.png', dpi=300, bbox_inches='tight')
plt.close()

evolucion_anual = df.groupby('año')['valor_unidad'].agg(['mean', 'std', 'min', 'max'])
print(evolucion_anual)

plt.figure(figsize=(12, 6))
plt.plot(evolucion_anual.index, evolucion_anual['mean'], marker='o')
plt.fill_between(evolucion_anual.index, evolucion_anual['mean'] - evolucion_anual['std'], evolucion_anual['mean'] + evolucion_anual['std'], alpha=0.2)
plt.title('Evolución anual (media ± desviación)')
plt.grid(True, alpha=0.3)
plt.savefig('data/graficas_comparativas/evolucion_anual.png', dpi=300, bbox_inches='tight')
plt.close()


## 15. Exportes finales y resumen

Guardamos el dataset consolidado, un resumen de limpieza y una versión lista para modelado.


In [ ]:
Path("data/processed").mkdir(parents=True, exist_ok=True)

df.to_csv("data/processed/pensiones_limpio_final.csv", index=False, encoding='utf-8')

resumen_limpieza = {
    'filas_finales': len(df),
    'columnas_finales': len(df.columns),
    'duplicados_eliminados': int(duplicados),
    'outliers_detectados': int(len(outliers)),
    'memoria_mb': df.memory_usage(deep=True).sum() / 1024**2,
    'fecha_limpieza': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}
pd.Series(resumen_limpieza).to_csv("data/processed/resumen_limpieza.csv")

df_modelado = df.dropna(subset=['valor_unidad', 'fecha', 'nombre_entidad', 'nombre_fondo']).copy()
df_modelado.to_csv("data/processed/pensiones_listo_modelado.csv", index=False, encoding='utf-8')
print("Archivos listos en data/processed/")
